## Imports

In [10]:
import cv2
import numpy as np
import sys
import math
import imutils
import matplotlib.pyplot as plt

## Globals & Prepare Model

---

Object Detection

---

In [11]:
#Import Object Detection Model Parameters
OBJ_config = "Model/yolov4.cfg"
OBJ_weights = "Model/yolov4.weights"
OBJ_class_names_file = "Model/coco.names"

# read class names from text file
OBJ_class_names = None
with open(OBJ_class_names_file, 'r') as f:
    OBJ_class_names = [line.strip() for line in f.readlines()]

OBJ_threshold = 0.5

---

Object Tracking

---

In [12]:
TRACK_list = cv2.legacy.MultiTracker_create()

---

Pose Estimation

---

In [13]:
#Import Pose Estimation Model Parameters
POSE_proto = "Model/pose_deploy_linevec.prototxt"
POSE_weights = "Model/pose_iter_440000.caffemodel"

POSE_body_parts = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_part_pairs = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

POSE_threshold = 0.2

In [14]:
#Create Models
OBJ_net = cv2.dnn.readNet(OBJ_config, OBJ_weights)
POSE_net = cv2.dnn.readNetFromCaffe(POSE_proto, POSE_weights)

#Set Models to use GPU
OBJ_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
OBJ_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)
POSE_net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
POSE_net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)


## Import DataSet

In [15]:
#Initialize the video stream
MEDIA_RAW = cv2.VideoCapture("Dataset/VideoDemo.mp4")

## Detect People

In [16]:
def get_output_layers(net):

    layer_names = net.getLayerNames()

    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

def detect_individuals_from_image(OBJ_img):

    OBJ_width = OBJ_img.shape[1]
    OBJ_height = OBJ_img.shape[0]
    OBJ_scale = 0.00392

    OBJ_blob = cv2.dnn.blobFromImage(OBJ_img, OBJ_scale, (416, 416), (0,0,0), True, crop=False)

    OBJ_net.setInput(OBJ_blob)
    outs = OBJ_net.forward(get_output_layers(OBJ_net))

    # initialization
    class_ids = []
    confidences = []
    boxes = []
    nms_threshold = 0.4

    # for each detection from each output layer
    # get the confidence, class id, bounding box params
    # and ignore weak detections (confidence < 0.5)
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > OBJ_threshold:
                center_x = int(detection[0] * OBJ_width)
                center_y = int(detection[1] * OBJ_height)
                w = int(detection[2] * OBJ_width)
                h = int(detection[3] * OBJ_height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    #Apply non-max suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, OBJ_threshold, nms_threshold)

    confident_people_box = []

    for i in indices:
        #Only return if detected object is a Person
        if class_ids[i] == 0:
            confident_people_box.append(boxes[i])

    return confident_people_box

## Track People

In [17]:
def start_tracking_from_boxes(TRACK_BBoxes, TRACK_img):

    #New Empty Track list
    new_TRACK_list = cv2.legacy.MultiTracker_create()

    #Fill new Track list with the new Bounding boxes
    for TRACK_box in TRACK_BBoxes:

        tracker = cv2.legacy.TrackerCSRT_create()
        new_TRACK_list.add(tracker, TRACK_img, TRACK_box)

    print("Start_Tracking NEW TRACK LIST")
    print(new_TRACK_list.getObjects())

    global TRACK_list
    TRACK_list = new_TRACK_list

    print("Start_Tracking FINAL TRACK LIST")
    print(TRACK_list.getObjects())
    print("------------------------------------------------")

def track_using_trackers(TRACK_img):

    global TRACK_list

    # grab the updated bounding box coordinates (if any) for each object that is being tracked
    (success, TRACK_boxes) = TRACK_list.update(TRACK_img)

    print("TRACK BOXES")
    print(TRACK_boxes)
    print("------------------------------------------------")

    TRACK_BOXES_count = 0

    # loop over the bounding boxes and draw them on the frame
    for box in TRACK_boxes:
        TRACK_BOXES_count += 1
        (x, y, w, h) = [int(v) for v in box]
        cv2.rectangle(TRACK_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(TRACK_img, str(TRACK_BOXES_count) , (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)

def update_trackers_boxes(TRACK_img):

    global TRACK_list

    TRACK_boxes = TRACK_list.getObjects()

    new_OBJ_boxes = detect_individuals_from_image(image)
    curr_to_new_Dict = {}
    new_TRACK_boxes = []

    for OBJ_box_i in range(len(new_OBJ_boxes)):
        new_OBJ_Center = get_center_of_box(new_OBJ_boxes[OBJ_box_i])
        cv2.circle(TRACK_img, new_OBJ_Center, 3, (255,0,0), 2)

        (x, y, w, h) = [int(v) for v in new_OBJ_boxes[OBJ_box_i]]
        cv2.rectangle(TRACK_img, (x, y), (x + w, y + h), (255, 0, 0), 2)

        distance = sys.maxsize
        trackerIndex = -1

        for TRACK_box_i in range(len(TRACK_boxes)):
            curr_TRACK_Center = get_center_of_box(TRACK_boxes[TRACK_box_i])
            cv2.circle(TRACK_img, curr_TRACK_Center, 3, (0,255,0), 2)

            curr_distance = math.dist(curr_TRACK_Center, new_OBJ_Center)

            if curr_distance < distance:
                distance = curr_distance
                trackerIndex = TRACK_box_i

        curr_to_new_Dict[trackerIndex] = new_OBJ_boxes[OBJ_box_i]

    for TRACK_box in TRACK_boxes:
        new_TRACK_boxes.append([int(TRACK_box[0]),int(TRACK_box[1]),int(TRACK_box[2]),int(TRACK_box[3])])

    for i in range(len(TRACK_boxes)):
        if i in curr_to_new_Dict:
            new_TRACK_boxes[i] = [int(curr_to_new_Dict[i][0]),int(curr_to_new_Dict[i][1]),int(curr_to_new_Dict[i][2]),int(curr_to_new_Dict[i][3])]

    print("TRACK")
    print(TRACK_boxes)
    print("OBJ")
    print(new_OBJ_boxes)
    print("NEW TRACK FROM OBJ")
    print(new_TRACK_boxes)
    print("------------------------------------------------")

    #Pass Bounding boxes to Tracker to start tracking
    #start_tracking_from_boxes(new_TRACK_boxes, TRACK_img)

def get_center_of_box(boundingBox):

    boundingBox_Xcenter = int((boundingBox[0] + (boundingBox[0] + boundingBox[2])) / 2)
    boundingBox_Ycenter = int((boundingBox[1] + (boundingBox[1] + boundingBox[3])) / 2)

    return(boundingBox_Xcenter, boundingBox_Ycenter)


## Main Pipeline

In [18]:
frameCount = 0
OBJ_boxes = []

#Loop over the frames from the video stream
while cv2.waitKey(1) < 0:

    #Grab the frame from the threaded video stream
    hasFrame, image = MEDIA_RAW.read()

    if not hasFrame:
        cv2.waitKey()
        cv2.destroyAllWindows()
        break

    frameCount += 1

    #Only Detect and Track People in the first frame
    if frameCount == 1:
        #Detect People in Image and return their Boxes Position
        OBJ_boxes = detect_individuals_from_image(image)
        #Pass Bounding boxes to Tracker to start tracking
        start_tracking_from_boxes(OBJ_boxes, image)

    #Sync trackers with new frame
    track_using_trackers(image)

    if frameCount%1 == 0:
        update_trackers_boxes(image)

    #show the output frame
    cv2.imshow("Frame", image)

MEDIA_RAW.release()

# close all windows
cv2.destroyAllWindows()

Start_Tracking NEW TRACK LIST
[[543.5 136.5 193.  379. ]
 [403.5 150.5 161.  409. ]
 [829.5 801.5 371.  283. ]
 [892.  526.5 270.  315. ]]
Start_Tracking FINAL TRACK LIST
[[543.5 136.5 193.  379. ]
 [403.5 150.5 161.  409. ]
 [829.5 801.5 371.  283. ]
 [892.  526.5 270.  315. ]]
------------------------------------------------
TRACK BOXES
[[544. 136. 193. 379.]
 [404. 150. 161. 409.]
 [830. 800. 371. 283.]
 [892. 526. 270. 315.]]
------------------------------------------------
TRACK
[[544. 136. 193. 379.]
 [404. 150. 161. 409.]
 [830. 800. 371. 283.]
 [892. 526. 270. 315.]]
OBJ
[[553.5, 143.0, 179, 370], [406.5, 159.0, 151, 400], [927.5, 530.0, 229, 276], [833.0, 694.0, 360, 392]]
NEW TRACK FROM OBJ
[[553, 143, 179, 370], [406, 159, 151, 400], [833, 694, 360, 392], [927, 530, 229, 276]]
------------------------------------------------
TRACK BOXES
[[544. 137. 193. 379.]
 [404. 149. 161. 409.]
 [830. 800. 371. 283.]
 [901. 536. 249. 291.]]
-----------------------------------------------